In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('pruthvi.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

28 documents
5 classes ['age', 'goodbye', 'greetings', 'hours', 'name']
44 unique lemmatized words ["'s", 'a', 'age', 'am', 'are', 'bye', 'call', 'can', 'cao', 'cya', 'day', 'going', 'good', 'goodbye', 'greeting', 'guy', 'have', 'hello', 'hey', 'hi', 'hour', 'how', 'i', 'is', 'it', 'later', 'leaving', 'me', 'name', 'of', 'old', 'open', 'operation', 'see', 'should', 'tell', 'up', 'what', 'whats', 'when', 'who', 'ya', 'you', 'your']


In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\Admin\AppData\Local\Temp\ipykernel_15988\2836083368.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
6/6 [==============================] - 1s 4ms/step - loss: 1.6432 - accuracy: 0.1786
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 1.5375 - accuracy: 0.2143
Epoch 3/200
6/6 [==============================] - 0s 7ms/step - loss: 1.5082 - accuracy: 0.3571
Epoch 4/200
6/6 [==============================] - 0s 6ms/step - loss: 1.4962 - accuracy: 0.3214
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 1.4923 - accuracy: 0.2857
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 1.4046 - accuracy: 0.5357
Epoch 7/200
6/6 [==============================] - 0s 6ms/step - loss: 1.3075 - accuracy: 0.5000
Epoch 8/200
6/6 [==============================] - 0s 6ms/step - loss: 1.1696 - accuracy: 0.6429
Epoch 9/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2527 - accuracy: 0.5000
Epoch 10/200
6/6 [==============================] - 0s 6ms/step - loss: 1.1184 - accuracy: 0.6071
Epoch 11/200
6/6 [===========

6/6 [==============================] - 0s 3ms/step - loss: 0.0625 - accuracy: 1.0000
Epoch 85/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0179 - accuracy: 1.0000
Epoch 86/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0424 - accuracy: 1.0000
Epoch 87/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0284 - accuracy: 1.0000
Epoch 88/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0166 - accuracy: 1.0000
Epoch 89/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0231 - accuracy: 1.0000
Epoch 90/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0588 - accuracy: 1.0000
Epoch 91/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0343 - accuracy: 1.0000
Epoch 92/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0253 - accuracy: 1.0000
Epoch 93/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0529 - accuracy: 1.0000
Epoch 94/200
6/6 [===============

6/6 [==============================] - 0s 3ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 168/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 169/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 170/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0096 - accuracy: 1.0000
Epoch 171/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 172/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0096 - accuracy: 1.0000
Epoch 173/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0285 - accuracy: 1.0000
Epoch 174/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0249 - accuracy: 1.0000
Epoch 175/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0433 - accuracy: 0.9643
Epoch 176/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0420 - accuracy: 0.9643
Epoch 177/200
6/6 [=====